In [1]:
import sys
sys.path.append('..')

In [2]:
import os
from pathlib import Path
import json
from tqdm import tqdm
import random
from collections import defaultdict

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertModel, BertTokenizer, BertConfig, AdamW, get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup
from data_utils import (
    load_dataset, 
    get_examples_from_dialogues, 
    convert_state_dict, 
    DSTInputExample, 
    OpenVocabDSTFeature, 
    DSTPreprocessor, 
    WOSDataset)
    
from inference import inference
from evaluation import _evaluation

In [3]:
import wandb
# !wandb login  # run once

In [4]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU        
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(42)

In [5]:
def increment_output_dir(output_path, exist_ok=False):
  path = Path(output_path)
  if (path.exists() and exist_ok) or (not path.exists()):
    return str(path)
  else:
    dirs = glob.glob(f"{path}*")
    matches = [re.search(rf"%s(\d+)" %path.stem, d) for d in dirs]
    i = [int(m.groups()[0]) for m in matches if m]
    n = max(i) + 1 if i else 2
    return f"{path}{n}"

### args setting

In [6]:
from argparse import Namespace

args = {
    'data_dir': 'data/train_dataset',
    'model_dir': 'trade',
    'train_batch_size': 16,
    'eval_batch_size': 32,
    'learning_rate': 1e-4,
    'adam_epsilon': 1e-8,
    'max_grad_norm': 1.0,   
    'num_train_epochs': 30,
    'warmup_ratio': 0.1,
    'random_seed': 42,
    'model_name_or_path': 'monologg/koelectra-base-v3-discriminator',
    'hidden_size': 768,
    'vocab_size': None,
    'hidden_dropout_prob': 0.1,
    'proj_dim': None,
    'teacher_forcing_ratio': 0.5,    
}

args = Namespace(**args)

In [62]:
# wandb sweep 생성 시 parameters에 전달하는 config 설정
# hyperparameter_defaults = dict(
#     batch_size = args.batch_size,
#     learning_rate = args.learning_rate,
#     epochs = args.num_train_epochs,
#     weight_decay = args.weight_decay,
#     attn_head = args.attn_head,
#     distance_metric = args.distance_metric,
    
#     dropout = 0.1,
#     smoothing = 0.2
#     model_name = 'BertForSequenceClassification',
#     tokenizer_name = 'BertTokenizer',
#     )

# wandb.init(config=hyperparameter_defaults, project="TRADE")
wandb.init(project="TRADE")
config = wandb.config

wandb: ERROR Problem finishing run
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 1532, in _atexit_cleanup
    self._on_finish()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 1668, in _on_finish
    self.history._flush()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_history.py", line 59, in _flush
    self._callback(row=self._data, step=self._step)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 836, in _history_callback
    row, step, publish_step=not_using_tensorboard
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 227, in publish_history
    self._publish_history(history)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 211, in _publish_history
    self._publish(rec)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 528, in _publis

wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


## Data loading

In [8]:
train_data_file = "/opt/ml/repo/taepd/input/data/train_dataset/train_dials.json"
slot_meta = json.load(open("/opt/ml/repo/taepd/input/data/train_dataset/slot_meta.json"))
ontology = json.load(open("/opt/ml/repo/taepd/input/data/train_dataset/ontology.json"))
train_data, dev_data, dev_labels = load_dataset(train_data_file)

# dialogue_level=False : SUMBT와 다르게 dialogue context level로 input하므로
train_examples = get_examples_from_dialogues(train_data,
                                             user_first=False,
                                             dialogue_level=False)
dev_examples = get_examples_from_dialogues(dev_data,
                                           user_first=False,
                                           dialogue_level=False)

100%|██████████| 699/699 [00:00<00:00, 2805.83it/s]


In [9]:
print(len(train_examples))
print(len(dev_examples))

46170
5075


## TRADE Preprocessor 

기존의 GRU 기반의 인코더를 BERT-based Encoder로 바꿀 준비를 합시다.

1. 현재 `_convert_example_to_feature`에서는 `max_seq_length`를 핸들하고 있지 않습니다. `input_id`와 `segment_id`가 `max_seq_length`를 넘어가면 좌측부터 truncate시키는 코드를 삽입하세요.

2. hybrid approach에서 얻은 교훈을 바탕으로 gate class를 3개에서 5개로 늘려봅시다.
    - WoS 데이터셋의 특성상 (boolean type이 많음) 'yes', 'no' gate를 추가했을 때 더 나은 성능을 보입니다.
    - `gating2id`를 수정하세요
    - 이에 따른 `recover_state`를 수정하세요.

3. word dropout을 구현하세요.

In [21]:
class TRADEPreprocessor(DSTPreprocessor):
    def __init__(
        self,
        slot_meta,
        src_tokenizer,
        trg_tokenizer=None,
        ontology=None,
        max_seq_length=512,
    ):
        self.slot_meta = slot_meta
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer if trg_tokenizer else src_tokenizer
        self.ontology = ontology
        self.gating2id =  {"none": 0, "dontcare": 1, "yes": 2, "no": 3, "ptr": 4}         
        self.id2gating = {v: k for k, v in self.gating2id.items()}
        self.max_seq_length = max_seq_length
        
    def _convert_example_to_feature(self, example):
        dialogue_context = " [SEP] ".join(example.context_turns + example.current_turn)

        input_id = self.src_tokenizer.encode(dialogue_context, add_special_tokens=False)
        max_length = self.max_seq_length - 2
        
        if len(input_id) > max_length:
            gap = len(input_id) - max_length
            input_id = input_id[gap:]
       
        input_id = (
            [self.src_tokenizer.cls_token_id]
            + input_id
            + [self.src_tokenizer.sep_token_id]
        )
        segment_id = [0] * len(input_id)  # bert기반이 되었을 땐 segment 구분이 필요할 수 있음

        target_ids = []
        gating_id = []
        if not example.label:
            example.label = []

        state = convert_state_dict(example.label)
        for slot in self.slot_meta:
            value = state.get(slot, "none")
            target_id = self.trg_tokenizer.encode(value, add_special_tokens=False) + [
                self.trg_tokenizer.sep_token_id
            ]  # [sep]를 추가하여, AutoRegressive시, EOS token으로 사용
            target_ids.append(target_id)
            gating_id.append(self.gating2id.get(value, self.gating2id["ptr"]))
        target_ids = self.pad_ids(target_ids, self.trg_tokenizer.pad_token_id)   # collate_fn에서 처리해주는 것 같은데?
        return OpenVocabDSTFeature(
            example.guid, input_id, segment_id, gating_id, target_ids
        )

    def convert_examples_to_features(self, examples):
        return list(map(self._convert_example_to_feature, examples))

    def recover_state(self, gate_list, gen_list):
        assert len(gate_list) == len(self.slot_meta)
        assert len(gen_list) == len(self.slot_meta)

        recovered = []
        for slot, gate, value in zip(self.slot_meta, gate_list, gen_list):
            if self.id2gating[gate] == "none":
                continue

            if self.id2gating[gate] == "dontcare":
                recovered.append("%s-%s" % (slot, "dontcare"))
                continue

            token_id_list = []
            for id_ in value:
                if id_ in self.trg_tokenizer.all_special_ids:
                    break

                token_id_list.append(id_)
            value = self.trg_tokenizer.decode(token_id_list, skip_special_tokens=True)

            if value == "none":
                continue

            recovered.append("%s-%s" % (slot, value))

        return recovered

    def collate_fn(self, batch):
        guids = [b.guid for b in batch]
        input_ids = torch.LongTensor(
            self.pad_ids([b.input_id for b in batch], self.src_tokenizer.pad_token_id)
        )
        segment_ids = torch.LongTensor(
            self.pad_ids([b.segment_id for b in batch], self.src_tokenizer.pad_token_id)
        )
        input_masks = input_ids.ne(self.src_tokenizer.pad_token_id)

        gating_ids = torch.LongTensor([b.gating_id for b in batch])
        target_ids = self.pad_id_of_matrix(
            [torch.LongTensor(b.target_ids) for b in batch],
            self.trg_tokenizer.pad_token_id,
        )
        return input_ids, segment_ids, input_masks, gating_ids, target_ids, guids

## Convert_Examples_to_Features 

In [22]:
tokenizer = BertTokenizer.from_pretrained('dsksd/bert-ko-small-minimal')
processor = TRADEPreprocessor(slot_meta, tokenizer, max_seq_length=512)

train_features = processor.convert_examples_to_features(train_examples)
dev_features = processor.convert_examples_to_features(dev_examples)

# 간이 테스트 시 사용
# train_features = processor.convert_examples_to_features(train_examples[:1000])
# dev_features = processor.convert_examples_to_features(dev_examples)

Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors


In [23]:
print(len(train_features))
print(len(dev_features))

46170
5075


# Model 

1. `GRUEncoder`를 `BertModel`로 교체하세요. 이에 따라 `tie_weight` 함수가 수정되어야 합니다.

In [24]:
class TRADE(nn.Module):
    def __init__(self, config, slot_vocab, slot_meta, pad_idx=0):
        super(TRADE, self).__init__()
        self.slot_meta = slot_meta
        # https://huggingface.co/dsksd/bert-ko-small-minimal/blob/main/config.json
        if config.model_name_or_path:
            self.encoder = BertModel.from_pretrained(config.model_name_or_path)
        else:
            self.encoder = BertModel(config)    
          
        self.decoder = SlotGenerator(
            config.vocab_size,
            config.hidden_size,
            config.hidden_dropout_prob,
            config.n_gate,
            None,
            pad_idx,
        )
        
        # init for only subword embedding
        self.decoder.set_slot_idx(slot_vocab)
        self.tie_weight()

    # encoder와 decoder의 embedding matrix sharing    
    def tie_weight(self):
#         self.decoder.embed = self.encoder.get_input_embeddings()
        self.decoder.embed.weight = self.encoder.embeddings.word_embeddings.weight
        if self.decoder.proj_layer:
            self.decoder.proj_layer.weight = self.encoder.proj_layer.weight

    def forward(self, 
                input_ids,  # p^history를 vocal size로 변환하기 위해 input_ids받음 
                token_type_ids,
                attention_mask=None, 
                max_len=10,  # maximum length를 target_id에서 받고, inference 때는 고정된 constant를 받음
                teacher=None):  # teacher forcing을 위한 GT Sequence

        encoder_outputs, pooled_output = self.encoder(input_ids=input_ids)
        all_point_outputs, all_gate_outputs = self.decoder(
            input_ids, encoder_outputs, pooled_output.unsqueeze(0), attention_mask, max_len, teacher
        )

        return all_point_outputs, all_gate_outputs
    
class SlotGenerator(nn.Module):
    def __init__(
        self, vocab_size, hidden_size, dropout, n_gate, proj_dim=None, pad_idx=0
    ):
        super(SlotGenerator, self).__init__()
        self.pad_idx = pad_idx
        self.vocab_size = vocab_size
        self.embed = nn.Embedding(
            vocab_size, hidden_size, padding_idx=pad_idx
        )  # shared with encoder

        if proj_dim:
            self.proj_layer = nn.Linear(hidden_size, proj_dim, bias=False)
        else:
            self.proj_layer = None
        self.hidden_size = proj_dim if proj_dim else hidden_size

        self.gru = nn.GRU(
            self.hidden_size, self.hidden_size, 1, dropout=dropout, batch_first=True
        )  # unidirectional GRU
        self.n_gate = n_gate
        self.dropout = nn.Dropout(dropout)
        self.w_gen = nn.Linear(self.hidden_size * 3, 1)  # p_gen을 만들 때 3가지를 concat하므로 *3
        self.sigmoid = nn.Sigmoid()
        self.w_gate = nn.Linear(self.hidden_size, n_gate)

    def set_slot_idx(self, slot_vocab_idx):
        whole = []
        max_length = max(map(len, slot_vocab_idx))   # tokenized slot_meta 길이 최댓값: ex)4
        for idx in slot_vocab_idx:
            if len(idx) < max_length:
                gap = max_length - len(idx)
                idx.extend([self.pad_idx] * gap)
            whole.append(idx)
        self.slot_embed_idx = whole  # torch.LongTensor(whole)  ex) [[6728, 6479, 1, 1],..]

    def embedding(self, x):
        x = self.embed(x)
        if self.proj_layer:
            x = self.proj_layer(x)
        return x

    def forward(
        self, input_ids, encoder_output, hidden, input_masks, max_len, teacher=None
    ):
        input_masks = input_masks.ne(1)
        # J, slot_meta : key : [domain, slot] ex> LongTensor([1,2])
        # J,2
        batch_size = encoder_output.size(0)
        slot = torch.LongTensor(self.slot_embed_idx).to(input_ids.device)  ##
        slot_e = torch.sum(self.embedding(slot), 1)  # J,d  J * slot_vocab_idx 를 embedding을 거쳐서 sum해주어 J * hidden 이 됨
        J = slot_e.size(0)

        all_point_outputs = torch.zeros(batch_size, J, max_len, self.vocab_size).to(  # max_len: max gen len
            input_ids.device
        )  # Output Tensor (for Placeholder)
        
        # Parallel Decoding
        w = slot_e.repeat(batch_size, 1).unsqueeze(1)
        hidden = hidden.repeat_interleave(J, dim=1)
        encoder_output = encoder_output.repeat_interleave(J, dim=0)
        input_ids = input_ids.repeat_interleave(J, dim=0)
        input_masks = input_masks.repeat_interleave(J, dim=0)
        for k in range(max_len):
            w = self.dropout(w)
            _, hidden = self.gru(w, hidden)  # 1,B,D

            # B,T,D * B,D,1 => B,T,1  (T가 |X_t|인듯, input sequence length)
            attn_e = torch.bmm(encoder_output, hidden.permute(1, 2, 0))  # B,T,1
            attn_e = attn_e.squeeze(-1).masked_fill(input_masks, -1e9)
            attn_history = F.softmax(attn_e, -1)  # B,T

            if self.proj_layer:
                hidden_proj = torch.matmul(hidden, self.proj_layer.weight)
            else:
                hidden_proj = hidden

            # B,D * D,V => B,V
            attn_v = torch.matmul(
                hidden_proj.squeeze(0), self.embed.weight.transpose(0, 1)
            )  # B,V
            attn_vocab = F.softmax(attn_v, -1)

            # B,1,T * B,T,D => B,1,D
            context = torch.bmm(attn_history.unsqueeze(1), encoder_output)  # B,1,D
            p_gen = self.sigmoid(
                self.w_gen(torch.cat([w, hidden.transpose(0, 1), context], -1))
            )  # B,1
            p_gen = p_gen.squeeze(-1)

            p_context_ptr = torch.zeros_like(attn_vocab).to(input_ids.device)
            p_context_ptr.scatter_add_(1, input_ids, attn_history)  # copy B,V
            p_final = p_gen * attn_vocab + (1 - p_gen) * p_context_ptr  # B,V
            _, w_idx = p_final.max(-1)

            if teacher is not None:
#                 w = self.embedding(teacher[:, :, k]).transpose(0, 1).reshape(batch_size * J, 1, -1)
                w = self.embedding(teacher[:, :, k]).reshape(batch_size * J, 1, -1)
            else:
                w = self.embedding(w_idx).unsqueeze(1)  # B,1,D
            if k == 0:
                gated_logit = self.w_gate(context.squeeze(1))  # B,3
                all_gate_outputs = gated_logit.view(batch_size, J, self.n_gate)
            all_point_outputs[:, :, k, :] = p_final.view(batch_size, J, self.vocab_size)

        return all_point_outputs, all_gate_outputs

# 모델 및 데이터 로더 정의

In [25]:
slot_vocab = []
for slot in slot_meta:
    slot_vocab.append(
        tokenizer.encode(slot.replace('-', ' '),  # 원래는 domain, slot을 영어와 달리 동일 길이로 토큰화되지 않기에 그냥 concat
                         add_special_tokens=False)
    )
    
config = BertConfig.from_pretrained('dsksd/bert-ko-small-minimal')
config.model_name_or_path = 'dsksd/bert-ko-small-minimal'
config.n_gate = len(processor.gating2id)
config.proj_dim = None
model = TRADE(config, slot_vocab, slot_meta)

wandb.watch(model)

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data = WOSDataset(train_features)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=4, sampler=train_sampler, collate_fn=processor.collate_fn,  num_workers=4)

dev_data = WOSDataset(dev_features)
dev_sampler = SequentialSampler(dev_data)
dev_loader = DataLoader(dev_data, batch_size=8, sampler=dev_sampler, collate_fn=processor.collate_fn, num_workers=4)

# Optimizer & Scheduler 선언

In [27]:
n_epochs = 100
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.01,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

t_total = len(train_loader) * n_epochs
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps=1e-8)
# scheduler = get_linear_schedule_with_warmup(
#     optimizer, num_warmup_steps=0.1, num_training_steps=t_total
# )


scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer, num_warmup_steps=0.1, num_training_steps=t_total, num_cycles=20
)
teacher_forcing = 0.5
model.to(device)

def masked_cross_entropy_for_value(logits, target, pad_idx=0):  # logits: (B, J, k, V)
    mask = target.ne(pad_idx)
    logits_flat = logits.view(-1, logits.size(-1))   # B*J*k, V (ex) [900, 35000])
    log_probs_flat = torch.log(logits_flat)
    target_flat = target.view(-1, 1)  # 열벡터로 변환  [900, 1]
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=target_flat)  # [900, 1]
    losses = losses_flat.view(*target.size())   # 행벡터로 변환 [900]
    losses = losses * mask.float()   # mask(padding)은 제외
    loss = losses.sum() / (mask.sum().float())
    return loss

loss_fnc_1 = masked_cross_entropy_for_value  # generation
loss_fnc_2 = nn.CrossEntropyLoss()  # gating

## Train

In [ ]:
# for checkpoint management
chk_list = []
output_dir = increment_output_dir(wandb.run.name)
PATIENCE = 0 
if not os.path.exists(f"checkpoint/{output_dir}"):
    os.makedirs(f"checkpoint/{output_dir}")  

best_score, best_checkpoint = 0, 0
epoch_miss_labels = defaultdict(list)
for epoch in range(n_epochs):
    batch_loss = []
    model.train()
    for step, batch in enumerate(tqdm(train_loader)):
        input_ids, segment_ids, input_masks, gating_ids, target_ids,_ = [b.to(device) if not isinstance(b, list) else b for b in batch]
        if teacher_forcing > 0.0 and random.random() < teacher_forcing:
            tf = target_ids
        else:
            tf = None

        all_point_outputs, all_gate_outputs = model(input_ids, segment_ids, input_masks, target_ids.size(-1))  # gt - length (generation)
        loss_1 = loss_fnc_1(all_point_outputs.contiguous(), target_ids.contiguous().view(-1))
        loss_2 = loss_fnc_2(all_gate_outputs.contiguous().view(-1, 5), gating_ids.contiguous().view(-1))
        loss = loss_1 + loss_2
        batch_loss.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if step % 100 == 0:                   
            print('[%d/%d] [%d/%d] %f' % (epoch+1, n_epochs, step, len(train_loader), loss.item()))
            # -- train 단계에서 Loss, Accuracy 로그 저장
            wandb.log({
                "train/loss": loss.item(),
                "train/gen_loss": loss_1.item(),
                "train/gate_loss": loss_2.item(),
                "train/epoch": epoch+1
            })
            
    predictions = inference(model, dev_loader, processor, device)
#     eval_result,  = _evaluation(predictions, dev_labels, slot_meta)
    eval_result, batch_miss_labels = _evaluation(predictions, dev_labels, slot_meta)
            
    epoch_miss_labels[epoch].extend(batch_miss_labels)   
    
    wandb.log({
            "eval/loss": loss.item(),
            "Joint Goal Accuracy": eval_result['joint_goal_accuracy'],
            "Turn Slot_Accuracy": eval_result['turn_slot_accuracy'],
            "Turn Slot F1": eval_result['turn_slot_f1']
            })
      
    for k, v in eval_result.items():
        print(f"{k}: {v}")
        
    if best_score < eval_result['joint_goal_accuracy']:
        print("Update Best checkpoint!")
        best_score = eval_result['joint_goal_accuracy']
        best_checkpoint = epoch
        if not os.path.isdir('./checkpoint'):
            os.makedirs('./checkpoint')
        output_path = f"checkpoint/{output_dir}/{epoch}_{step}_{best_score}.pth"
        chk_list.append(output_path)

        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, output_path)
        PATIENCE = 0
    else:
        PATIENCE += 1
        if PATIENCE == 1:
            break

#     torch.save(model.state_dict(), f"{args.model_dir}/model-{epoch}.bin")
#     torch.save(model.state_dict(), f"checkpoint/trage/{output_dir}/model-{epoch}.bin")
# print(f"Best checkpoint: {args.model_dir}/model-{best_checkpoint}.bin")    

  0%|          | 2/11543 [00:00<1:35:06,  2.02it/s]

[1/100] [0/11543] 0.000571


  1%|          | 102/11543 [00:19<34:37,  5.51it/s]

[1/100] [100/11543] 0.002938


  2%|▏         | 202/11543 [00:35<29:23,  6.43it/s]

...

 52%|█████▏    | 6002/11543 [17:14<17:34,  5.25it/s]

[1/100] [6000/11543] 0.004014


 53%|█████▎    | 6102/11543 [17:30<16:19,  5.56it/s]

[1/100] [6100/11543] 0.000153


 53%|█████▎    | 6115/11543 [17:33<16:18,  5.55it/s]

## Inference 

In [29]:
eval_data = json.load(open(f"/opt/ml/repo/taepd/input/data/eval_dataset/eval_dials.json", "r"))

eval_examples = get_examples_from_dialogues(
    eval_data, user_first=False, dialogue_level=False
)

# Extracting Featrues
eval_features = processor.convert_examples_to_features(eval_examples)
eval_data = WOSDataset(eval_features)
eval_sampler = SequentialSampler(eval_data)
eval_loader = DataLoader(
    eval_data,
    batch_size=8,
    sampler=eval_sampler,
    collate_fn=processor.collate_fn,
)

100%|██████████| 2000/2000 [00:00<00:00, 16252.49it/s]


In [43]:
predictions = inference(model, eval_loader, processor, device)

100%|██████████| 1847/1847 [05:46<00:00,  5.33it/s]


In [44]:
json.dump(predictions, open('predictions.csv', 'w'), indent=2, ensure_ascii=False)

### eda용 missing label exporting

In [45]:
with open('miss_labels.json', 'w') as outfile:
    json.dump(epoch_miss_labels, outfile)

In [48]:
# config = BertConfig.from_pretrained('dsksd/bert-ko-small-minimal')
# config.model_name_or_path = 'dsksd/bert-ko-small-minimal'
# config.n_gate = len(processor.gating2id)
# config.proj_dim = None
# model1 = TRADE(config, slot_vocab, slot_meta)
# PATH = 'checkpoint/golden-forest-14/3_11542_0.7749753694581281.pth'

# checkpoint = torch.load(PATH)
# model1.load_state_dict(checkpoint['model_state_dict'])

# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<All keys matched successfully>

In [49]:
# predictions = inference(model, eval_loader, processor, device)

100%|██████████| 1847/1847 [05:27<00:00,  5.63it/s]


In [50]:
# json.dump(predictions, open('predictions.csv', 'w'), indent=2, ensure_ascii=False)